In [2]:
import pandas as pd
from pandas import json_normalize

import requests 
import json
import os
from dotenv import load_dotenv 
from bs4 import BeautifulSoup
import re
import time


In [213]:


url = "https://food-nutrition-information.p.rapidapi.com/foods/search"

querystring = {"query":"rice"}

headers = {
    "X-RapidAPI-Key": "f5bc8265eemshffa75f77ba1b130p15dd15jsnf1b9068994ab",
"X-RapidAPI-Host": "food-nutrition-information.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)
sug = response.json()


In [214]:


url = "https://food-nutrition-information.p.rapidapi.com/food/356554"

headers = {
"X-RapidAPI-Key": "f5bc8265eemshffa75f77ba1b130p15dd15jsnf1b9068994ab",
"X-RapidAPI-Host": "food-nutrition-information.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers)

print(response.text)

KeyboardInterrupt: 

In [ ]:
coffee = response.json()
json_normalize(coffee)

# Web scraping for minimum wage in Turkey throughout the years


In [3]:
url = 'https://countryeconomy.com/national-minimum-wage/turkey'
res = requests.get(url)
html = res.content
soup = BeautifulSoup(html, "html.parser")

In [4]:
y = soup.select("td.fecha")
y[0].getText().strip()

year = [i.getText() for i in y]
# year = [i for i in year if 'June' not in i]
year[:3]

['June 2022', 'January 2022', 'January 2021']

In [5]:
mw = soup.find_all("td", attrs={"class":"numero"})
mw[0].getText().strip()

min_wage = [i.getText() for i in mw]
min_wage = [i for i in min_wage if '$' not in i and '€' not in i]

min_wage[:3]

['6,471.0', '5,004.0', '3,577.5']

In [6]:
mwu = soup.select("td.numero.dol")
mwu[0].getText().strip()

min_wage_usd = [i.getText() for i in mwu]
min_wage_usd[:3]

['$394.5', '$375.1', '$404.2']

In [24]:
turkey = {
    'year': year,
    'min_wage': min_wage,
    'min_wage_dollar': min_wage_usd
}

turkey_wages = pd.DataFrame(turkey)
turkey_wages = turkey_wages[turkey_wages['year'].str.contains('June')==False]


turkey_wages = turkey_wages.replace('January ', "", regex = True).replace('$', '', regex = True)
turkey_wages = turkey_wages.astype({'min_wage' : float, 'min_wage_dollar' : float}, errors = 'ignore')

turkey_wages.sample(3)

,year,min_wage,min_wage_dollar
21,2009,666.0,$447.4
5,2018,"2,029.5",$420.3
7,2016,"1,647.0",$545.3


# Cleaning the price variation dataset and merging the two

In [12]:
# Reading the dataset used for this anylisis.
# "global_prices" is a dataset displaying the price variation for different products in 76 countries from 2002 to 2021.

global_prices = pd.read_csv(".\Data\global_food_prices.csv", encoding='unicode_escape')
global_prices.sample(5)

C:\Users\mayar\AppData\Local\Temp\ipykernel_23916\1891304889.py:4: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  global_prices = pd.read_csv(".\Data\global_food_prices.csv", encoding='unicode_escape')


,adm0_id,adm0_name,adm1_id,adm1_name,mkt_id,mkt_name,cm_id,cm_name,cur_id,cur_name,pt_id,pt_name,um_id,um_name,mp_month,mp_year,mp_price,mp_commoditysource
1958459,257.0,United Republic of Tanzania,115007,Njombe,4553,Njombe Region,870,Beans (spotted) - Retail,0.0,TZS,15,Retail,5,KG,10,2019,2000.0000,NaN
1678632,999.0,State of Palestine,900008,Gaza,2007,Gaza,362,Oil (maize) - Retail,0.0,NIS,15,Retail,77,3 L,1,2017,20.5000,NaN
2022149,270.0,Zambia,3431,North-Western,1461,Zambezi,67,Maize (white) - Retail,0.0,ZMW,15,Retail,5,KG,12,2012,1.1111,NaN
418577,57.0,Colombia,935,Antioquia,842,Medellin,56,Maize (local) - Wholesale,0.0,COP,14,Wholesale,5,KG,1,2021,1027.0000,NaN
770392,118.0,Iraq,1572,NaN,1493,Baghdad,66,Beans (white) - Retail,0.0,IQD,15,Retail,5,KG,4,2020,2333.3333,NaN


In [13]:
# Creating a subset of the price variation that only takes into consideration Turkey.

prices_turkey = global_prices.loc[global_prices['adm0_name'] == "Turkey"]

#prices_turkey = prices_turkey.loc[prices_turkey['adm1_name'] == 'Buenos Aires D.f.']

In [14]:
# Since this analysis is focusing on the consumer, the prices for wholesale are not relevant.

prices_turkey = prices_turkey[prices_turkey['pt_name'].str.contains('Wholesale')==False]

prices_turkey = prices_turkey.replace(' - Retail', "", regex=True)


In [15]:

# Cleaning the table by dropping columns that are not relevant for the analysis and renaming the other ones 
# for readability and clarity.

prices_turkey.drop(['cur_id', 'cur_name', 'pt_id', 'um_id', 'mp_commoditysource', 'adm1_name'], axis = 'columns', inplace=True)

prices_turkey.drop(['cm_id', 'adm0_id', 'adm1_id', 'mkt_id', 'mkt_name', 'pt_name'], axis = 'columns', inplace=True)

prices_turkey.columns = prices_turkey.columns.str.replace('mp_', "")

prices_turkey.sample(5)



,adm0_name,cm_name,um_name,month,year,price
1871829,Turkey,"Milk (powder, infant formula)",KG,2,2021,151.7855
1875578,Turkey,Cauliflower,KG,2,2020,3.9015
1873389,Turkey,Bulgur,KG,1,2017,3.3586
1875928,Turkey,Transport (public),Course,8,2018,3.1529
1872572,Turkey,Garlic,KG,4,2020,67.8587


In [16]:
# # Cleaning the avg_salaries_argentina dataframe and getting only the salaries in Buenos Aires.
# # The column 'fecha' is split into year, month and day so I can later merge the two dataframes by using
# # these columns as the common denominator (since the global_prices df has these columns already).

# avg_salaries_argentina = avg_salaries_argentina.loc[avg_salaries_argentina['zona_prov'] == 'BUENOS AIRES']

# avg_salaries_argentina[['year', 'month', 'day']] = avg_salaries_argentina['fecha'].str.split("-", expand=True)

# avg_salaries_argentina.drop(['fecha', 'day', 'zona_prov'], axis = 'columns', inplace=True)


# # Transforming the columns type to int 

# avg_salaries_argentina[['year', 'month']] = avg_salaries_argentina[['year', 'month']].apply(pd.to_numeric)

# avg_salaries_argentina.sample(3)



In [21]:
# Merging both dataframes by the year and creating a new one called products_wage_turkey that has information
# about the price variation of certain goods as well as the variation of the minimum wage in Turkey.

products_wage_turkey = prices_turkey.merge(turkey_wages, how='inner', on='year')
products_wage_turkey.sample(3)

,adm0_name,cm_name,um_name,month,year,price,min_wage,min_wage_dollar
7528,Turkey,Bread (common),KG,7,2020,6.2503,"2,943.0",$419.9
4976,Turkey,Oil (olive),L,9,2018,31.7553,"2,029.5",$420.3
8660,Turkey,Oil (olive),L,9,2020,31.2937,"2,943.0",$419.9


In [25]:
# More cleaning for readability (renaming columns and rounding values).

products_wage_turkey.rename(
    columns =
    {'cm_name':'product', 'adm0_name':'country','um_name':'unit'},
    inplace = True
)

products_wage_turkey['price'] = products_wage_turkey['price'].round(2)


products_wage_turkey.sample(2)


,country,product,unit,month,year,price,min_wage,min_wage_dollar
5435,Turkey,Onions,KG,11,2019,1.68,"2,558.4",$450.9
2996,Turkey,Cucumbers,KG,6,2017,1.62,"1,777.5",$487.2


In [26]:
products_wage_filtered = products_wage_turkey.loc[products_wage_turkey.groupby(['year', 'product'])['price'].idxmin()]
products_wage_filtered.sort_values(by = ['product', 'year'])

,country,product,unit,month,year,price,min_wage,min_wage_dollar
2958,Turkey,Apples,KG,1,2017,2.48,"1,777.5",$487.2
5076,Turkey,Apples,KG,1,2018,3.08,"2,029.5",$420.3
6138,Turkey,Apples,KG,3,2019,3.55,"2,558.4",$450.9
8234,Turkey,Apples,KG,1,2020,3.73,"2,943.0",$419.9
9617,Turkey,Apples,KG,1,2021,5.03,"3,577.5",$404.2
...,...,...,...,...,...,...,...,...
3073,Turkey,Zucchini,KG,5,2017,1.91,"1,777.5",$487.2
5191,Turkey,Zucchini,KG,7,2018,2.71,"2,029.5",$420.3
6253,Turkey,Zucchini,KG,7,2019,1.83,"2,558.4",$450.9
8349,Turkey,Zucchini,KG,7,2020,2.15,"2,943.0",$419.9
